# Homework 9
## Kincaid & Cheney Section 7.3
### Problem 21 (a)

If we use the standard basis $\{1, x, x^2\}$ on the vector space of polynomial of degree at most 2, then the following computations should yield the same thing.

First, let's compute the integrals:
\begin{align*}
\int_{-1}^1\,dx &= 2 \\
\int_{-1}^1 x\,dx &= 0\\
\int_{-1}^1 x^2\,dx &=\frac{2}{3}
\end{align*}

These should agree with our results from Gaussian quadrature.  So in particular:
\begin{align*}
2 &= A + B + C \\
0 &= -\sqrt{\frac{3}{5}}A + 0 + \sqrt{\frac{3}{5}}C \\
\frac{2}{3} &= \frac{3}{5}A + 0 + \frac{3}{5} C
\end{align*}

Let's solve this system in Macaulay2:

In [8]:
A = matrix {
    {1, 1, 1},
    {-sqrt(3/5), 0, sqrt(3/5)},
    {3/5, 0, 3/5}}
b = vector(RR, {2, 0, 2/3})
solve(A, b)


o8 = | .555556 |
     | .888889 |
     | .555556 |

         3
o8 : RR
       53


That looks like it's probably $\begin{pmatrix}A\\B\\C\end{pmatrix} = \dfrac{1}{9}\begin{pmatrix}5 \\ 8 \\ 5\end{pmatrix}$.  Let's check:

In [9]:
A * 1/9 * vector {5, 8, 5}


o9 = |    2    |
     |    0    |
     | .666667 |

         3
o9 : RR
       53


Seems right!  So we have:

\begin{equation*}
\int_{-1}^1f(x)\,dx \approx \frac{5}{9}f\left(-\sqrt{\frac{3}{5}}\right) + \frac{8}{9}f(0)+\frac{5}{9}f\left(\sqrt{\frac{3}{5}}\right) 
\end{equation*}

### Problem 22
Let's use $u$-substituion.  Given some integral $\displaystyle{\int_a^bf(x)\,dx}$, we'd like to find some one-to-one function $g$ for which $g(a) = -1$ and $g(b) = 1$.  A linear function would work beautifully.  And in particular, such a function would have slope $\frac{1-(-1)}{b-a} = \frac{2}{b-a}$, giving us
\begin{equation*}
g(x) = -1 + \frac{2}{b-a}(x - a)
\end{equation*}

So if $u = g(x)$, then
\begin{align*}
\frac{2}{b-a}(x - a) &= u + 1 \\
x &= \frac{b-a}{2}(u + 1) + a \\
dx &= \frac{b-a}{2}\,du
\end{align*}

So, via $u$-substitution, we have
\begin{align*}
\int_a^b f(x)\,dx &= \int_{-1}^1f\left(\frac{b-a}{2}(u + 1) + a\right)\cdot\frac{b-a}{2}\,du \\
&\approx\frac{b-a}{2}\left[\frac{5}{9}f\left(\frac{b-a}{2}\left(1-\sqrt{\frac{3}{5}}\right) + a\right) + \frac{8}{9}f\left(\frac{b-a}{2} + a\right)+\frac{5}{9}f\left(\frac{b-a}{2}\left(1+\sqrt{\frac{3}{5}}\right) + a\right) \right]\\
\end{align*}

Let's go ahead and write the Macaulay2 code we can use for parts (a) and (b):

In [10]:
gaussianQuadrature = (f, a, b) -> (
    h := (b - a)/2;  -- we use this a bunch, so might as well make things a bit simpler
    h * (5/9*f(h * (1 - sqrt(3/5)) + a) + 8/9*f(h + a) + (5/9*f(h * (1 + sqrt(3/5)) + a))))


o10 = gaussianQuadrature

o10 : FunctionClosure


#### Part (a)
Now let's use our code.  Note that the `identity` function in Macaulay2 is just a shortcut for $f(x) = x$.

In [11]:
gaussianQuadrature(identity, 0, pi/2)


o11 = 1.23370055013617

o11 : RR (of precision 53)


Let's check:

\begin{equation*}
\int_0^{\pi/2}x\,dx = \frac{1}{2}\left(\frac{\pi}{2}\right)^2
\end{equation*}

In [12]:
1/2*(pi/2)^2


o12 = 1.23370055013617

o12 : RR (of precision 53)


It's exact!  Of course, that's to be expected since we were integrating a degree 1 polynomial.

#### Part (b)
Using our code:

In [13]:
gaussianQuadrature(t -> sin t / t, 0, 4)


o13 = 1.758022025436357

o13 : RR (of precision 53)


This is a nonelementary integral, so we can't check our work exactly.  However, this particular integral is common in applications, and the **sine integral** function is actually defined as
\begin{equation*}
\text{Si}(x) = \int_0^x\frac{\sin t}{t}\,dt
\end{equation*}

So let's compute $\text{Si}(4)$ to check our work.  The sine integral function isn't really something algebraic geometers care about too much, so it's not implemented natively in Macaulay2.  But we can compute it using Macaulay2's Python interface.  The Python module SciPy has a function `scipy.special.sici` that computes values of both Si and Ci (the *cosine integral* function).

In [19]:
needsPackage "Python"
scipyspecial = import "scipy.special"
scipyspecial@@sici 4


o19 = (1.758203138949053, -0.1409816978869305)

o19 : PythonObject of class tuple


The first element of this tuple is what we're looking for.  We were very close!

## Kincaid & Cheney Section 7.4
### Problem 1

We start with Equation (6), the [Euler-Maclaurin formula](https://en.wikipedia.org/wiki/Euler%E2%80%93Maclaurin_formula), which says that for some $\xi_0\in(0,1)$,

\begin{equation*}
\int_0^1f(t)\,dt = \frac{1}{2}(f(0) + f(1)) + \sum_{k=1}^{m-1}A_{2k}\left(f^{(2k-1)}(0) - f^{(2k-1)}(0)\right) - A_{2m}f^{(2m)}(\xi_0),
\end{equation*}
where $(2k)!A_{2k}=B_{2k}$, a [Bernoulli number](https://en.wikipedia.org/wiki/Bernoulli_number).

Now pick $x_i,x_{i+1}$ and define $g(t) = f(x_i + ht)$, where $h=x_{i+1}-x_i$.  So, applying the Euler-Maclaurin formula to $g$, we have
\begin{align*}
\int_0^1g(t)\,dt &= \frac{1}{2}(g(0) + g(1)) + \sum_{k=1}^{m-1}A_{2k}\left(g^{(2k-1)}(0) - g^{(2k-1)}(0)\right) - A_{2m}g^{(2m)}(\xi_0),
\end{align*}

By inductively applying the chain rule, we see that $g^{(n)}(t) = h^nf^{(n)}(x_i + ht)$ for any nonnegative integer $h$.  Since $x_i + h\cdot 0 = x_i$ and $x_i + h\cdot 1 = x_{i+1}$, it follows that
\begin{align*}
\int_0^1f(x_i+ht)\,dt &= \frac{1}{2}(f(x_i) + f(x_{i+1})) + \sum_{k=1}^{m-1}A_{2k}h^{2k-1}\left(f^{(2k-1)}(x_i) - f^{(2k-1)}(x_{i+1})\right) - A_{2m}h^{2m}f^{(2m)}(x_i + h\xi_0),
\end{align*}

Define $\xi_i$ to be the argument in the last term, i.e., $\xi=x_i + h\xi_0$.  Now let's use $u$-substitution (actually, $x$-subtitution) to transform the left-hand side with $x = x_i + ht$.  Then $dx = h\,dt$, or $dt = \frac{1}{h}\,dx$.  This gives us
\begin{align*}
\int_{x_i}^{x_{i+1}}\frac{1}{h}f(x)\,dx &= \frac{1}{2}(f(x_i) + f(x_{i+1})) + \sum_{k=1}^{m-1}A_{2k}h^{2k-1}\left(f^{(2k-1)}(x_i) - f^{(2k-1)}(x_{i+1})\right) - A_{2m}h^{2m}f^{(2m)}(\xi_i) \\
\int_{x_i}^{x_{i+1}}f(x)\,dx &= \frac{h}{2}(f(x_i) + f(x_{i+1})) + \sum_{k=1}^{m-1}A_{2k}h^{2k}\left(f^{(2k-1)}(x_i) - f^{(2k-1)}(x_{i+1})\right) - A_{2m}h^{2m+1}f^{(2m)}(\xi_i),
\end{align*}
which is Equation (7), as desired.

### Problem 2
Now we sum up Equation (7) over all $i\in\{0,\ldots,2^n-1\}$, giving us
\begin{equation*}
\sum_{i=0}^{2^n-1}\int_{x_i}^{x_{i+1}}f(x)\,dx = \sum_{i=0}^{2^n-1}\left[\frac{h}{2}(f(x_i) + f(x_{i+1})) + \sum_{k=1}^{m-1}A_{2k}h^{2k}\left(f^{(2k-1)}(x_i) - f^{(2k-1)}(x_{i+1})\right) - A_{2m}h^{2m+1}f^{(2m)}(\xi_i)\right]
\end{equation*}
If we define $a = x_0$ and $b = x_{2^n}$, then we can combine the left-hand side into a single integral, giving us
\begin{equation*}
\int_a^bf(x)\,dx = \sum_{i=0}^{2^n-1}\left[\frac{h}{2}(f(x_i) + f(x_{i+1})) + \sum_{k=1}^{m-1}A_{2k}h^{2k}\left(f^{(2k-1)}(x_i) - f^{(2k-1)}(x_{i+1})\right) - A_{2m}h^{2m+1}f^{(2m)}(\xi_i)\right]
\end{equation*}

Let's focus on the middle term on the right-hand side for a moment for a fixed $k$.  We have
\begin{align*}
A_{2k}h^{2k}\sum_{i=0}^{2^n-1}\left(f^{(2k-1)}(x_i) - f^{(2k-1)}(x_{i+1})\right) &= A_{2k}h^{2k}\left(\sum_{i=0}^{2^n-1}f^{(2k-1)}(x_i) - \sum_{i=0}^{2^n-1}f^{(2k-1)}(x_{i+1})\right) \\
\end{align*}
Shifting the indices on the second sum, this gives us
\begin{equation*}
= A_{2k}h^{2k}\left(\sum_{i=0}^{2^n-1}f^{(2k-1)}(x_i) - \sum_{i=1}^{2^n}f^{(2k-1)}(x_i)\right)\\
\end{equation*}
Now we can split out the $i=0$ case from the first sum and the $i=2^n$ case from the second sum, but combine all the others, which end up canceling i.e.,
\begin{align*}
&= A_{2k}h^{2k}\left(f^{(2k-1)}(x_0) + \sum_{i=1}^{2n-1}(f^{(2k-1)}(x_i) - f^{(2k-1)}(x_i)) - f^{(2k-1)}(x_{2^n})\right) \\
&= A_{2h}h^{2k}\left(f^{(2k-1)}(a) - f^{(2k-1)}(b)\right) \\
\end{align*}

Putting this back in to the larger equation, we have
\begin{equation*}
\int_a^bf(x)\,dx = \frac{h}{2}\sum_{i=0}^{2^n-1}(f(x_i) + f(x_{i+1})) + \sum_{k=1}^{m-1}A_{2h}h^{2k}\left(f^{(2k-1)}(a) - f^{(2k-1)}(b)\right) - A_{2m}h^{2m+1}\sum_{i=0}^{2^n-1}f^{(2m)}(\xi_i)
\end{equation*}

It remains to deal with the last term, i.e., to prove the claim (the "in particular, justify ..." statement in the problem description) that there exists some $\xi$ for which
\begin{equation*}
h^{2m+1}\sum_{i=0}^{2^n-1}f^{(2m)}(\xi_i) = (b - a)h^{2m}f^{(2m)}(\xi)
\end{equation*}

Since we are assuming $f\in C^{2m}[a,b]$, we know that $f^{(2m)}$ is continuous.  In other words, it obeys the intermediate value theorem.

Let $c$ and $d$ be the bounds on $f^{(2m)}$ in the interval $[a,b]$.  In other words, we can say that $c\leq f^{(2m)}(x)\leq d$ for all $x\in[a,b]$.  This implies that
\begin{align*}
2^nc &\leq \sum_{i=0}^{2^n-1}f^{(2m)}(\xi_i) \leq 2^n d \\
c &\leq \frac{1}{2^n}\sum_{i=0}^{2^n-1}f^{(2m)}(\xi_i) \leq d \\
\end{align*}
And since $h=\frac{b-a}{2^n}$, or $\frac{1}{2^n} = \frac{h}{b-a}$, we may rewrite this as
\begin{equation*}
c \leq\frac{h}{b-a} \sum_{i=0}^{2^n-1}f^{(2m)}(\xi_i) \leq d
\end{equation*}

So by the IVT, it follows that there exists some $\xi\in[a,b]$ such that
\begin{align*}
\frac{h}{b-a}\sum_{i=0}^{2^n-1}f^{(2m)}(\xi_i) &= f(\xi) \\
h\sum_{i=0}^{2^n-1}f^{(2m)}(\xi_i) &= (b-a)f(\xi) \\
\end{align*}
If we multiply both sides by $h^{2m}$, then we see that this proves our claim.

### Richardson Extrapolation

#### Equation (5)
Using $R$ instead of $D$ and $m$ instead of $k$, we get from Equation (19) on p. 474 the following:
\begin{align*}
R(n, m) &= \frac{4^m}{4^m - 1}R(n, m - 1) - \frac{1}{4^k - 1}R(n - 1, m - 1) \\
&= \left(\frac{4^m - 1}{4^m - 1} + \frac{1}{4^m - 1}\right)R(n, m - 1) - \frac{1}{4^k - 1}R(n - 1, m - 1) \\
&= R(n, m - 1) + \frac{1}{4^m - 1}\left(R(n, m - 1) - R(n - 1, m - 1)\right),
\end{align*}
giving us Equation (5)

#### Section 7.1 Theorem 1

Let's rephrase Theorem 1 using our notation for this section.  Again, its $D$ is our $R$ and its $k$ is our $m$.  Note that $L$ is the integral we're estimating, and the previous arguments stemming from the Maclaurin formula have put it in the desired form (see p. 472 Equation (16)).  So letting $h = b - a$, Theorem 1 gives us:
\begin{align*}
R(n, m - 1) &= \int_a^b f(x)\,dx + \sum_{j = m}^\infty A_{jm}\left(\frac{b - a}{2^n}\right)^{2j} \\
\int_a^b f(x)\,dx &= R(n, m - 1) - \sum_{j = m}^\infty A_{jm}\left(\frac{b - a}{2^n}\right)^{2j} \\
\end{align*}
This is true for all $m$, and in particular if we replace $m - 1$ with $m$.  Let's also simplify by letting $h = \frac{b-a}{2^n}$ (this is the $h$ from p. 505 and not the $h$ from the statement of Theorem 1), giving us
\begin{align*}
\int_a^b f(x)\,dx &= R(n, m) - \sum_{j = m + 1}^\infty A_{j,m + 1}h^{2j} \\
&= R(n, m) - \sum_{j = 0}^\infty A_{j + m + 1,m + 1}h^{2(j + m + 1)} \\
&= R(n, m) - h^{2m + 2}\sum_{j = 0}^\infty A_{j + m + 1,m + 1}h^{2j} \\
&= R(n, m) + O(h^{2m + 2}),
\end{align*}
actually slightly stronger than stated on p. 505.

### Computer Problem 1
First, let's load our Romberg integration function from class:


In [14]:
romberg = (f, a, b, M) -> (
    R := new MutableHashTable;
    h := b - a;
    R#(0, 0) = (b-a)/2*(f a + f b);
    for n from 1 to M do (
        h /= 2;
        R#(n, 0) = 1/2*R#(n - 1, 0) + h * sum(
            1..2^(n-1), i -> f(a + (2*i - 1)*h));
        for m from 1 to n do (
            R#(n, m) = R#(n, m-1) + (
                R#(n, m-1) - R#(n-1, m-1))/(4^m - 1)));
    matrix table(M, M, (n, m) -> if m <= n then R#(n,m) else 0))


o14 = romberg

o14 : FunctionClosure


#### Part (a)

In [13]:
romberg(x -> if x == 0 then 1 else sin x / x, 0, 1, 7)


o13 = | .920735 0       0       0       0       0       0       |
      | .939793 .946146 0       0       0       0       0       |
      | .944514 .946087 .946083 0       0       0       0       |
      | .945691 .946083 .946083 .946083 0       0       0       |
      | .945985 .946083 .946083 .946083 .946083 0       0       |
      | .946059 .946083 .946083 .946083 .946083 .946083 0       |
      | .946077 .946083 .946083 .946083 .946083 .946083 .946083 |

                 7         7
o13 : Matrix RR    <-- RR
               53        53


Let's check our work.  This is the *sine integral* function again, so we can check by computing $\text{Si}(1)$ using Python as in Homework 8.

In [20]:
needsPackage "Python"
scipy = import "scipy"
scipy@@special@@sici 1


o20 = (0.9460830703671831, 0.33740392290096816)

o20 : PythonObject of class tuple


#### Part (b)
Note that this function is undefined at 0 again, and we have, using L'Hôpital's rule,
\begin{equation*}
\lim_{x\to 0}\frac{\cos x - e^x}{\sin x} = \lim_{x\to 0}\frac{-\sin x - e^x}{\cos x} = \frac{0 - 1}{1} = -1
\end{equation*}
So we can make this function continuous by using $-1$ at 0.

In [22]:
romberg(x -> if x == 0 then -1 else (cos x - exp x)/sin x, -1, 1, 7)


o22 = | -2.79321 0        0        0        0        0        0        |
      | -2.3966  -2.2644  0        0        0        0        0        |
      | -2.28522 -2.24809 -2.247   0        0        0        0        |
      | -2.25633 -2.2467  -2.2466  -2.2466  0        0        0        |
      | -2.24903 -2.2466  -2.24659 -2.24659 -2.24659 0        0        |
      | -2.2472  -2.24659 -2.24659 -2.24659 -2.24659 -2.24659 0        |
      | -2.24674 -2.24659 -2.24659 -2.24659 -2.24659 -2.24659 -2.24659 |

                 7         7
o22 : Matrix RR    <-- RR
               53        53


### Part (c)

As suggested, let's first use the substitution $x = \frac{1}{t}$, or, equivalently $t = \frac{1}{x}$.  Then $dx = -\frac{1}{t^2}\,dt$, giving us
\begin{align*}
\int_1^\infty(xe^x)^{-1}\,dx &= \lim_{b\to\infty}\int_1^b (xe^x)^{-1}\,dx \\
&= -\lim_{b\to\infty}\int_{1/1}^{1/b}\left(\frac{1}{t}e^{1/t}\right)^{-1}\cdot\frac{1}{t^2}\,dt \\
&= -\int_1^0\frac{1}{te^{1/t}}\,dt \\
&= \int_0^1\frac{1}{te^{1/t}}\,dt
\end{align*}

By L'Hôpital's rule again, it can be shown that the function approaches 0 from the right.

In [23]:
romberg(t -> if t == 0 then 0 else 1/(t*exp(1/t)), 0, 1, 7)


o23 = | .18394  0       0       0       0       0       0       |
      | .227305 .24176  0       0       0       0       0       |
      | .219834 .217344 .215716 0       0       0       0       |
      | .219351 .21919  .219313 .21937  0       0       0       |
      | .219384 .219394 .219408 .21941  .21941  0       0       |
      | .219384 .219384 .219383 .219383 .219383 .219383 0       |
      | .219384 .219384 .219384 .219384 .219384 .219384 .219384 |

                 7         7
o23 : Matrix RR    <-- RR
               53        53


This number is actually $-\text{Ei}(-1)$, where Ei is the [exponential integral function](https://en.wikipedia.org/wiki/Exponential_integral).  We can again check our work in Python:

In [24]:
-scipy@@special@@expi(-1)


o24 = 0.21938393439552062

o24 : PythonObject of class numpy.float64
